In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, sinh, cosh, sqrt, pi
from scipy import constants

# Физические константы
hbar = 1.0545718e-34  # (Дж·с)
eV = 1.60218e-19  # (Дж)
m_e = 9.10938356e-31  # (кг) масса электрона

class KronigPenneyModel:
    """Класс для моделирования зонной структуры по модели Кронига-Пенни"""
    
    def __init__(self, a, b, U0):
        """
        Parameters:
        a: ширина потенциальной ямы (м)
        b: ширина потенциального барьера (м)  
        U0: высота потенциального барьера (Дж)
        """
        self.a = a
        self.b = b
        self.U0 = U0
        self.period = a + b  # период решётки
        
        # Параметр модели
        self.P = (m_e * U0 * a * b) / (2 * hbar**2)
    
    def dispersion_relation(self, E):
        """Уравнение дисперсионного соотношения Кронига-Пенни"""
        kappa = sqrt(2 * m_e * E) / hbar if E >= 0 else 0
        
        if E < self.U0:
            # Случай E < U0
            beta = sqrt(2 * m_e * (self.U0 - E)) / hbar
            term1 = (beta**2 - kappa**2) / (2 * kappa * beta) * sinh(beta * self.b) * sin(kappa * self.a)
            term2 = cosh(beta * self.b) * cos(kappa * self.a)
            return term1 + term2
        else:
            # Случай E >= U0
            beta = sqrt(2 * m_e * (E - self.U0)) / hbar
            term1 = (kappa**2 + beta**2) / (2 * kappa * beta) * sin(beta * self.b) * sin(kappa * self.a)
            term2 = cos(beta * self.b) * cos(kappa * self.a)
            return term1 + term2
    
    def find_energy_bands(self, E_max, n_points=1000):
        """Нахождение разрешённых энергетических зон"""
        E_values = np.linspace(1e-9, E_max, n_points)
        f_values = np.array([self.dispersion_relation(E) for E in E_values])
        
        # Разрешённые зоны: |f(E)| <= 1
        allowed_mask = np.abs(f_values) <= 1
        
        # Группируем непрерывные разрешённые зоны
        bands = []
        in_band = False
        band_start = 0
        
        for i, is_allowed in enumerate(allowed_mask):
            if is_allowed and not in_band:
                band_start = i
                in_band = True
            elif not is_allowed and in_band:
                band_end = i - 1
                bands.append((E_values[band_start], E_values[band_end]))
                in_band = False
        
        if in_band:
            bands.append((E_values[band_start], E_values[-1]))
        
        return bands, E_values, f_values
    
    def calculate_band_structure(self, k_points=500):
        """Расчёт зонной структуры E(k)"""
        k_values = np.linspace(-pi/self.period, pi/self.period, k_points)
        E_bands = []
        
        for k in k_values:
            # Для каждого k решаем уравнение cos(k*(a+b)) = f(E)
            def equation(E):
                return self.dispersion_relation(E) - cos(k * self.period)
            
            # Находим корни уравнения для данного k
            E_roots = []
            E_search = np.linspace(1e-9, 10 * self.U0, 1000)
            f_search = np.array([equation(E) for E in E_search])
            
            for i in range(len(E_search) - 1):
                if f_search[i] * f_search[i+1] <= 0:
                    # Есть корень в этом интервале
                    root = 0.5 * (E_search[i] + E_search[i+1])
                    E_roots.append(root)
            
            E_bands.append(E_roots)
        
        return k_values, E_bands

def analyze_extreme_cases():
    """Анализ крайних случаев"""
    
    # Случай 1: Свободный электрон (U0 = 0)
    print("Случай 1: Свободный электрон (U0 = 0)")
    kp_free = KronigPenneyModel(a=1e-10, b=1e-10, U0=0)
    bands_free, E_vals_free, f_vals_free = kp_free.find_energy_bands(5 * eV)
    print(f"Количество разрешённых зон: {len(bands_free)}")
    
    # Случай 2: Непроницаемые стенки (U0 → ∞)
    print("\nСлучай 2: Непроницаемые стенки (U0 → ∞)")
    kp_wall = KronigPenneyModel(a=1e-10, b=1e-10, U0=1e6 * eV)
    bands_wall, E_vals_wall, f_vals_wall = kp_wall.find_energy_bands(5 * eV)
    print(f"Количество разрешённых зон: {len(bands_wall)}")
    
    # Промежуточные случаи
    print("\nПромежуточные случаи:")
    U0_values = [0.1 * eV, 1 * eV, 10 * eV]
    for U0 in U0_values:
        kp = KronigPenneyModel(a=1e-10, b=1e-10, U0=U0)
        bands, _, _ = kp.find_energy_bands(5 * eV)
        print(f"U0 = {U0/eV:.1f} эВ: {len(bands)} зон")
    
    return kp_free, kp_wall, E_vals_free, f_vals_free, E_vals_wall, f_vals_wall

def plot_dispersion_relations():
    """Построение графиков дисперсионных соотношений"""
    
    # Параметры кристалла
    a = 2e-10  # ширина ямы
    b = 1e-10  # ширина барьера
    
    # Разные значения высоты барьера
    U0_values = [0, 0.5 * eV, 2 * eV, 10 * eV]
    colors = ['blue', 'green', 'orange', 'red']
    labels = ['Свободный электрон', 'Слабый барьер', 'Средний барьер', 'Сильный барьер']
    
    plt.figure(figsize=(15, 10))
    
    for i, (U0, color, label) in enumerate(zip(U0_values, colors, labels)):
        kp = KronigPenneyModel(a, b, U0)
        
        # Дисперсионное соотношение
        plt.subplot(2, 2, 1)
        E_max = 5 * eV
        E_vals = np.linspace(1e-9, E_max, 1000)
        f_vals = np.array([kp.dispersion_relation(E) for E in E_vals])
        
        plt.plot(E_vals/eV, f_vals, color=color, label=label, linewidth=2)
        plt.axhline(1, color='black', linestyle='--', alpha=0.5)
        plt.axhline(-1, color='black', linestyle='--', alpha=0.5)
        
        # Зонная структура
        plt.subplot(2, 2, i+2)
        k_vals, E_bands = kp.calculate_band_structure()
        
        for band_idx in range(min(5, len(E_bands[0]))):
            E_band = [band[band_idx] if band_idx < len(band) else np.nan for band in E_bands]
            plt.plot(k_vals * kp.period / pi, np.array(E_band)/eV, 
                    color=color, linewidth=2)
        
        plt.xlabel('k (π/(a+b))')
        plt.ylabel('E (эВ)')
        plt.title(f'Зонная структура: {label}')
        plt.grid(True)
    
    plt.subplot(2, 2, 1)
    plt.xlabel('E (эВ)')
    plt.ylabel('f(E)')
    plt.title('Дисперсионное соотношение Кронига-Пенни')
    plt.legend()
    plt.grid(True)
    plt.ylim(-3, 3)
    
    plt.tight_layout()
    plt.show()

def plot_band_gaps():
    """Анализ ширины запрещённых зон"""
    
    a = 2e-10
    b = 1e-10
    
    # Зависимость от высоты барьера
    U0_range = np.logspace(-2, 2, 50) * eV  # от 0.01 до 100 эВ
    band_gaps = []
    
    for U0 in U0_range:
        kp = KronigPenneyModel(a, b, U0)
        bands, _, _ = kp.find_energy_bands(5 * eV)
        
        gaps = []
        for i in range(len(bands) - 1):
            gap = bands[i+1][0] - bands[i][1]  # начало след. зоны - конец предыдущей
            if gap > 0:
                gaps.append(gap)
        
        band_gaps.append(gaps[0]/eV if gaps else 0)  # ширина первой запрещённой зоны
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.semilogx(U0_range/eV, band_gaps, 'b-', linewidth=2)
    plt.xlabel('Высота барьера U0 (эВ)')
    plt.ylabel('Ширина запрещённой зоны (эВ)')
    plt.title('Зависимость ширины запрещённой зоны от высоты барьера')
    plt.grid(True)
    
    # Зависимость от отношения a/b
    plt.subplot(1, 2, 2)
    ratio_range = np.linspace(0.1, 10, 50)
    gaps_vs_ratio = []
    
    for ratio in ratio_range:
        a_local = ratio * 1e-10
        b_local = 1e-10
        kp = KronigPenneyModel(a_local, b_local, 2 * eV)
        bands, _, _ = kp.find_energy_bands(5 * eV)
        
        gaps = []
        for i in range(len(bands) - 1):
            gap = bands[i+1][0] - bands[i][1]
            if gap > 0:
                gaps.append(gap)
        
        gaps_vs_ratio.append(gaps[0]/eV if gaps else 0)
    
    plt.plot(ratio_range, gaps_vs_ratio, 'r-', linewidth=2)
    plt.xlabel('Отношение a/b (ширина ямы/ширина барьера)')
    plt.ylabel('Ширина запрещённой зоны (эВ)')
    plt.title('Зависимость ширины запрещённой зоны от отношения a/b')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    print("Модель Кронига-Пенни: Анализ зонной структуры кристалла")
    
    # Анализ крайних случаев
    kp_free, kp_wall, E_free, f_free, E_wall, f_wall = analyze_extreme_cases()
    
    # Построение графиков
    plot_dispersion_relations()
    plot_band_gaps()
    
    print("\nАнализ завершён!")